# TensorFlow to Keras

### Reference
- https://github.com/myutwo150/keras-inception-resnet-v2

### Pre-trained tensorflow models

- https://github.com/davidsandberg/facenet

In [1]:
!git clone https://github.com/davidsandberg/facenet

Cloning into 'facenet'...
remote: Enumerating objects: 3149, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3149 (delta 0), reused 1 (delta 0), pack-reused 3146
Receiving objects: 100% (3149/3149), 2.92 MiB | 19.30 MiB/s, done.
Resolving deltas: 100% (2233/2233), done.


In [26]:
import os
import re
import numpy as np
import tensorflow as tf
import tf_slim

import sys
sys.path.append('../code/')
from facenet.src.models.inception_resnet_v1 import *

In [16]:
tf_model_dir = '/content/model/tf/'
npy_weights_dir = '/content/model/keras/npy_weights/'
weights_dir = '/content/model/keras/weights/'
model_dir = '/content/model/keras/model/'

weights_filename = 'facenet_keras_weights.h5'
model_filename = 'facenet_keras.h5'

In [17]:
os.makedirs(npy_weights_dir, exist_ok=True)
os.makedirs(weights_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

In [10]:
# regex for renaming the tensors to their corresponding Keras counterpart
re_repeat = re.compile(r'Repeat_[0-9_]*b')
re_block8 = re.compile(r'Block8_[A-Za-z]')

def get_filename(key):
    filename = str(key)
    filename = filename.replace('/', '_')
    filename = filename.replace('InceptionResnetV1_', '')

    # remove "Repeat" scope from filename
    filename = re_repeat.sub('B', filename)

    if re_block8.match(filename):
        # the last block8 has different name with the previous 5 occurrences
        filename = filename.replace('Block8', 'Block8_6')

    # from TF to Keras naming
    filename = filename.replace('_weights', '_kernel')
    filename = filename.replace('_biases', '_bias')

    return filename + '.npy'


def extract_tensors_from_checkpoint_file(filename, output_folder):
    reader =  tf.compat.v1.train.NewCheckpointReader(filename)

    for key in reader.get_variable_to_shape_map():
        # not saving the following tensors
        if key == 'global_step':
            continue
        if 'AuxLogit' in key:
            continue

        # convert tensor name into the corresponding Keras layer weight name and save
        path = os.path.join(output_folder, get_filename(key))
        arr = reader.get_tensor(key)
        np.save(path, arr)

In [12]:
import gdown

file_id = '1EXPBSXwTaqrSC0OhUdXNmKSh9qJUQ55-'
output_file = 'model.zip'

gdown.download(f'https://drive.google.com/uc?id={file_id}', output_file, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1EXPBSXwTaqrSC0OhUdXNmKSh9qJUQ55-
From (redirected): https://drive.google.com/uc?id=1EXPBSXwTaqrSC0OhUdXNmKSh9qJUQ55-&confirm=t&uuid=f5bbd803-05d7-454c-8b82-369ba32e711f
To: /content/model.zip
100%|██████████| 192M/192M [00:01<00:00, 133MB/s]


'model.zip'

In [13]:
!unzip model.zip
!cp /content/20180402-114759/model-20180402-114759.ckpt-275.data-00000-of-00001 /content/model/tf
!cp /content/20180402-114759/20180402-114759.pb /content/model/tf
!cp /content/20180402-114759/model-20180402-114759.ckpt-275.index /content/model/tf
!cp /content/20180402-114759/model-20180402-114759.meta /content/model/tf


Archive:  model.zip
   creating: 20180402-114759/
  inflating: 20180402-114759/model-20180402-114759.meta  
  inflating: 20180402-114759/20180402-114759.pb  
  inflating: 20180402-114759/model-20180402-114759.ckpt-275.data-00000-of-00001  
  inflating: 20180402-114759/model-20180402-114759.ckpt-275.index  


In [19]:
extract_tensors_from_checkpoint_file(tf_model_dir+'model-20180402-114759.ckpt-275.index', npy_weights_dir)

In [23]:
input_shape = (160,160,3);
input_layer = tf.keras.layers.Input(shape=input_shape)

In [27]:
model = inception_resnet_v1(input_layer)
# model.summary()

AttributeError: module 'tensorflow' has no attribute 'variable_scope'

In [ ]:
print('Loading numpy weights from', npy_weights_dir)
for layer in model.layers:
    if layer.weights:
        weights = []
        for w in layer.weights:
            weight_name = os.path.basename(w.name).replace(':0', '')
            weight_file = layer.name + '_' + weight_name + '.npy'
            weight_arr = np.load(os.path.join(npy_weights_dir, weight_file))
            weights.append(weight_arr)
        layer.set_weights(weights)

print('Saving weights...')
model.save_weights(os.path.join(weights_dir, weights_filename))
print('Saving model...')
model.save(os.path.join(model_dir, model_filename))

Loading numpy weights from ../model/keras/npy_weights/
Saving weights...
Saving model...
